In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Implementing RNN

class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b # h_(t-1) * W_h + X_t * W_t = h_t
        h_next = np.tahh(t) # tanh for activation

        self.cache = (x, h_prev, h_next)
        return h_next